In [141]:
import numpy as np
import torch

cyt = 5
efficiency = 1.0  # Efficiency of converting muscle <-> cyt
rads = np.array([-1,2])   # single cell's muscles
# masses = np.abs(rads)**2 * np.sign(radi)
rad_deltas = np.array([1,1])
activate_muscle_growth = np.vectorize(lambda rads, rad_deltas: (rads + rad_deltas)**2 - rads**2)
csa_deltas = activate_muscle_growth(rads, rad_deltas) # cross-sectional area
positive_csa_deltas = csa_deltas[csa_deltas > 0]
negative_csa_deltas = csa_deltas[csa_deltas < 0]

print("Radii: ", rads)
print("CSA Mags: ", rads**2)
print("Radial Deltas: ", rad_deltas)
print("CSA Deltas: ", csa_deltas)
print("Cyt: ", cyt)

print("-------")

cyt -= sum(negative_csa_deltas) * efficiency
new_csa_mags = rads**2.0
new_csa_mags[csa_deltas < 0] += negative_csa_deltas
print("New CSA Mags: ", new_csa_mags)
print("New Cyt: ", cyt)

print("-------")

cyt_desired = sum(positive_csa_deltas) # before efficiency loss
csa_delta_distribution = positive_csa_deltas / cyt_desired

if cyt_desired > cyt:
    cyt_consumed = cyt
else:
    cyt_consumed = cyt_desired

csa_grown = cyt_consumed * efficiency
new_csa_mags[csa_deltas > 0] += csa_grown * csa_delta_distribution

cyt -= cyt_consumed

new_rad_mags = np.sqrt(new_csa_mags)
new_signs = np.sign(rads + rad_deltas)

print("Cyt Desired: ", cyt_desired)
print("New CSA Mags: ", new_csa_mags)
print("New Rads: ", new_rad_mags * new_signs)
print("New Cyt: ", cyt)



Radii:  [-1  2]
CSA Mags:  [1 4]
Radial Deltas:  [1 1]
CSA Deltas:  [-1  5]
Cyt:  5
-------
New CSA Mags:  [0. 4.]
New Cyt:  6.0
-------
Cyt Desired:  5
New CSA Mags:  [0. 9.]
New Rads:  [0. 3.]
New Cyt:  1.0


In [140]:
activate_muscle_growth = np.vectorize(lambda rads, rad_deltas: (rads + rad_deltas)**2 - rads**2)
def grow_muscle(rads, rad_deltas, cyt, efficiency):
    csa_deltas = activate_muscle_growth(rads, rad_deltas) # cross-sectional area
    positive_csa_deltas = csa_deltas[csa_deltas > 0]
    negative_csa_deltas = csa_deltas[csa_deltas < 0]

    cyt -= sum(negative_csa_deltas) * efficiency
    new_csa_mags = rads**2.0
    new_csa_mags[csa_deltas < 0] += negative_csa_deltas

    cyt_desired = sum(positive_csa_deltas) # before efficiency loss
    csa_delta_distribution = positive_csa_deltas / cyt_desired

    if cyt_desired > cyt:
        cyt_consumed = cyt
    else:
        cyt_consumed = cyt_desired

    csa_grown = cyt_consumed * efficiency
    new_csa_mags[csa_deltas > 0] += csa_grown * csa_delta_distribution

    cyt -= cyt_consumed

    new_rad_mags = np.sqrt(new_csa_mags)
    new_signs = np.sign(rads + rad_deltas)

    return new_rad_mags * new_signs, cyt

cyt = 5
efficiency = 1.0  # Efficiency of converting muscle <-> cyt
rads = np.array([-1,2])   # single cell's muscles
# masses = np.abs(rads)**2 * np.sign(radi)
rad_deltas = np.array([1,1])
grow_muscle(rads, rad_deltas, cyt, efficiency)

Radii:  [-1  2]
CSA Mags:  [1 4]
Radial Deltas:  [1 1]
CSA Deltas:  [-1  5]
Cyt:  5
-------
New CSA Mags:  [0. 4.]
New Cyt:  6.0
-------
Cyt Desired:  5
New CSA Mags:  [0. 9.]
New Rads:  [0. 3.]
New Cyt:  1.0


(array([0., 3.]), 1.0)

In [142]:
import torch

def activate_muscle_growth(rads, rad_deltas):
    return (rads + rad_deltas)**2 - rads**2

def grow_muscle(rads, rad_deltas, cyt, efficiency):
    csa_deltas = activate_muscle_growth(rads, rad_deltas)
    positive_csa_deltas = csa_deltas[csa_deltas > 0]
    negative_csa_deltas = csa_deltas[csa_deltas < 0]

    cyt -= torch.sum(negative_csa_deltas) * efficiency
    new_csa_mags = rads**2.0
    new_csa_mags[csa_deltas < 0] += negative_csa_deltas

    cyt_desired = torch.sum(positive_csa_deltas)
    csa_delta_distribution = positive_csa_deltas / cyt_desired

    cyt_consumed = torch.where(cyt_desired > cyt, cyt, cyt_desired)

    csa_grown = cyt_consumed * efficiency
    new_csa_mags[csa_deltas > 0] += csa_grown * csa_delta_distribution

    cyt -= cyt_consumed

    new_rad_mags = torch.sqrt(new_csa_mags)
    new_signs = torch.sign(rads + rad_deltas)

    return new_rad_mags * new_signs, cyt

# Define your batches
rads_batch = torch.tensor([[0.2, 2, -3, 0.1, 0.1], [0.3, 2.5, -2, 0.2, 0.2], [0.4, 3, -1, 0.3, 0.3]])
rad_deltas_batch = torch.tensor([[0.1, -0.2, 1, 3, 0.1], [0.2, -0.3, 2, 2, 0.2], [0.3, -0.4, 3, 1, 0.3]])
cyt_batch = torch.tensor([2, 3, 4])
efficiency_batch = torch.tensor([0.8, 0.85, 0.9])

rads_batch = torch.tensor([[0.2, 2, -3, 0.1, 0.1], [0.3, 2.5, -2, 0.2, 0.2], [0.4, 3, -1, 0.3, 0.3]])
rad_deltas_batch = torch.tensor([[0.1, -0.2, 1, 3, 0.1], [0.2, -0.3, 2, 2, 0.2], [0.3, -0.4, 3, 1, 0.3]])
cyt_batch = torch.tensor([2, 3, 4])
efficiency_batch = torch.tensor([0.8, 0.85, 0.9])

# Call the function with batched inputs
new_rads_batch, new_cyt_batch = grow_muscle(rads_batch, rad_deltas_batch, cyt_batch, efficiency_batch)

RuntimeError: result type Float can't be cast to the desired output type Long